In [1]:
# REF:: https://youtu.be/SBNE5UbGGH4
import pandas as pd
import numpy as np
import xlsxwriter
from xlsxwriter.utility import xl_range_abs

# Setup

## helper functions

### get_data()

In [2]:
def get_data(rows = 200, year=2020, seed=42):
    '''
    Generate sales product test data for given year:
    location, product, month, target_sales, target_profit, actual_sales, actual profit.
    
    Example
    -------
    df = get_data()

    
    Parameters
    ----------
    number_of_rows - int - number of records/rows required.
    
    year - int - sales year to generate data for.
    
    
    Returns
    -------
    Pandas Dataframe
    
    '''
    np.random.seed(seed)
    
    locations = ['London', 'Paris', 'Milan']
    products = ['Tops & Blouses', 'Jeans', 'Footwear', 'Beachwear', 'Sportswear']  

    data = {
        'location': np.random.choice(locations, size=rows),
        'product': np.random.choice(products, size=rows),
        'month': np.random.choice(range(1, 13), size=rows),
        'target_sales': np.random.randint(14000, 40000, size=rows),
        'target%_profit': np.random.randint(10, size=rows) * .02
    }

    df = pd.DataFrame(data)

    df['month'] = df['month'].apply(
        lambda x: pd.Period(f'{year}-{str(x).zfill(2)}'))

    df['target_profit'] = df['target_sales'] * df['target%_profit']

    actual_sales_lambda = lambda x: x + (x * np.random.choice(range(-10, 10)) / 100)
    df['actual_sales'] = df['target_sales'].apply(actual_sales_lambda)

    df['actual_profit'] = (df['actual_sales'] * df['target%_profit']).round(2)
    df.drop(columns=['target%_profit'], inplace=True)
    df.month = pd.PeriodIndex(df.month).to_timestamp()
    
    df = df.sort_values(['month', 'location', 'product'])
    df.reset_index(drop=True, inplace=True)

    return df

### doughnut_chart()

In [3]:
def doughnut_chart(wb, category_range, value_range):
    '''
    Define doughnut chart

    Parameters:
    -----------
    wb: workbook - excel workbook reference
    
    category_range: str - indicates absolute excel range e.g. '$A$1:$A$40'
                        defining the data used for category
                        
    value_range: str - indicates absolute excel range e.g. '$A$1:$A$40'
                        defining the data used for value
                        
    Returns
    -------
    chart object (reference)                        
    '''  
    chart = wb.add_chart({'type': 'doughnut'})
    
    mycolors = ['#E6B9B8', '#7F7F7F', '#E17179', '#FFFFA3']
    points = [{'fill': {'color': color}} for color in mycolors]

    chart.add_series({
        'categories': category_range,
        'values': value_range,
        'data_labels': {'value': True, 'num_format': '#,##0'},
        'points': points
    })

    chart.set_hole_size(30)
    chart.set_plotarea({'layout': {'x': 0.01, 'y': 0.01, 'width': 0.87, 'height': 0.97}})
    chart.set_chartarea({'border': {'none': True}})

    return chart

### bar_chart()

In [4]:
def bar_chart(wb, category_range, value_range):
    '''
    Define bar chart

    Parameters:
    -----------
    wb: workbook - excel workbook reference
    
    category_range: str - indicates absolute excel range e.g. '$A$1:$A$40'
                        defining the data used for category
                        
    value_range: str - indicates absolute excel range e.g. '$A$1:$A$40'
                        defining the data used for value
                        
    Returns
    -------
    chart object (reference)                        
    '''  
    chart = wb.add_chart({'type': 'bar'})
    
    chart.add_series({
        'categories': category_range,
        'values': value_range,
        'data_labels': {'value': True, 'num_format': '#,##0'},
        'fill': {'color':'#9999FF'},
        'gap': 20
    })

    chart.set_legend({'none': True})
    chart.set_plotarea({'layout': {'x': 0.01, 'y': 0.01, 'width': 0.97, 'height': 0.67}})
    chart.set_chartarea({'border': {'none': True}})

    return chart

### column_chart()

In [5]:
def column_chart(wb, category_range, value_range):
    '''
    Define column chart

    Parameters:
    -----------
    wb: workbook - excel workbook reference
    
    category_range: str - indicates absolute excel range e.g. '$A$1:$A$40'
                        defining the data used for category
                        
    value_range: str - indicates absolute excel range e.g. '$A$1:$A$40'
                        defining the data used for value
                        
    Returns
    -------
    chart object (reference)                       
    '''  
    chart = wb.add_chart({'type': 'column'})
    
    chart.add_series({
        'categories': category_range,
        'values': value_range,
        'data_labels': {'value': True, 'num_format': '#,##0', 'font': {'rotation': -90}},
        'fill': {'color':'#604A7B'},
        'gap': 20
    })

    chart.set_legend({'none': True})
    
    chart.set_x_axis({'major_gridlines': {'visible': False}, 'num_format': '#,##0'})
    chart.set_y_axis({'visible': False, 'major_gridlines': {'visible': False}})
    
    chart.set_plotarea({'layout': {'x': 0.01, 'y': 0.01, 'width': 0.97, 'height': 0.67}})
    chart.set_chartarea({'border': {'none': True}})

    return chart

### calc_xl_range()

In [6]:
def calc_xl_range(sheet_name, first_row, column, last_row, header=True):
    '''
    Calculate Excel range
    
    Example:
    --------    
    calc_xl_range('Sheet1', 2, 1, 10, False)
    >> range: Sheet1!$B$3:$B$11

    
    Parameters:
    -----------
    sheet_name: str - sheet name
    
    first_row: int - first row
    
    column: int - column
    
    last_row: int - last row
    
    header: boolean - default True - take account of header
                    (add 1 to first/last rows)
    
    
    Notes:
    ------
    Uses -> xl_range_abs(first_row, first_col, last_row, last_col)

    '''
    if header:
        first_row += 1
        
    range_ = xl_range_abs(first_row, column, last_row, column)
    range_ = f'{sheet_name}!{range_}'     
#     print(f'range: {range_}')
    
    return range_  

# Sales data

## generate and import

In [7]:
df0 = get_data(rows=500)
df0.head(8)

,location,product,month,target_sales,target_profit,actual_sales,actual_profit
0,London,Beachwear,2020-01-01,39164,3916.40,36422.52,3642.25
1,London,Beachwear,2020-01-01,25383,4061.28,22844.70,3655.15
2,London,Beachwear,2020-01-01,16356,327.12,16356.00,327.12
3,London,Footwear,2020-01-01,17430,3137.40,18650.10,3357.02
4,London,Footwear,2020-01-01,34819,6267.42,33774.43,6079.40
5,London,Footwear,2020-01-01,25718,4629.24,27261.08,4906.99
6,London,Footwear,2020-01-01,39003,7020.54,39393.03,7090.75
7,London,Jeans,2020-01-01,23105,2310.50,24029.20,2402.92


## Summary (3 dataframes)

In [8]:
df1 = (df0.groupby('location')
          .agg(total_sales=('actual_sales', 'sum'))
          .reset_index())
# display(df1.head(3))

mask = "(location == 'Paris') & (month == month.max())"
df2 = (df0.query(mask)
          .groupby('product')
          .agg(total_sales=('actual_sales', 'sum'))
          .reset_index())
# display(df2.head(3))

df3 = (df0.groupby('product')
          .agg(total_sales=('actual_sales', 'sum'))
          .reset_index())
# display(df3.head(3))

## Excel export

In [9]:
xl_output = 'outputs/QuarterlyReport.xlsx'
sheet_name = 'Corporate'

with pd.ExcelWriter(xl_output) as writer:

    wb = writer.book
    
    ''' STEP 1:: Write data sheet with summary dataframes info
    
       To reduce repetitive code, define & iterate tuple containing:
        - each dataframe reference 
        - a tuple containing the starting row and column position
          to write the data into the worksheet
    '''
    tuple_sheets = (df1, (2, 1)), (df2, (2, 4)), (df3, (2, 7))
    
    for df, rowcol in tuple_sheets:
        row, col = rowcol
        df.to_excel(writer, sheet_name=sheet_name, 
                    startrow=row, startcol=col,
                    index=False, header=True)       

    ws = writer.sheets[sheet_name]
    ws.set_column('A:S', 12)

    
    ''' STEP 2 :: For each dataframe written to first worksheet,
                  generate chart object in the 'Report' worksheet
        
        To reduce repetitive code, define a list of tuples containing:
        - the target chart location
        - the function to call to create the chart
        - a tuple containing x and y chart scaling       
    '''
    ws2 = wb.add_worksheet('Report')
      
    chartlocations = [('B2', doughnut_chart, (.8, .7)), 
                      ('B18', bar_chart, (.8, .7)),
                      ('I12', column_chart, (.7, .7))]
    
    for idx, (dataframe, rowcol) in enumerate(tuple_sheets):

        xl_row, xl_col = rowcol # Excel (row, column) position of the data
        df_row, df_col = dataframe.shape # actual rows and columns in the dataframe

        category_range = calc_xl_range(sheet_name,  xl_row, xl_col, xl_row + df_row)      
        value_range = calc_xl_range(sheet_name,  xl_row, xl_col+1, xl_row + df_row)

        chart_location, chart_function, xy_scale = chartlocations[idx]        
        chart = chart_function(wb, category_range, value_range)
        
        x_scale, y_scale = xy_scale
        ws2.insert_chart(chart_location, chart, {'x_scale': x_scale, 'y_scale': y_scale})  

print(f'Written: {xl_output}')

Written: outputs/QuarterlyReport.xlsx
